In [28]:
from functools import reduce
from pathlib import Path

import pandas as pd

from penguinscrolls.util import get_penguin_dataset

In [3]:
dataset = get_penguin_dataset()

In [4]:
df = dataset.select_columns(
    ['input_md5', 'dataset', 'split', 'token_count']
).to_pandas()

In [29]:
eval_result_dir = './eval_result/' # evaluation result json files goes here

In [43]:
data = {
    p.stem:pd.read_json(p, lines=True) for p in Path(eval_result_dir).iterdir() if p.is_file() and p.suffix == '.json'
}
data = {
    k:v[v.score.notnull()] for k,v in data.items()
}
common_keys = reduce(lambda acc, x: acc & x, map(set, map(set, [v.input_md5 for v in data.values()])))

data = {k:v[v.input_md5.isin(common_keys)][['input_md5', 'score']].rename(
    columns={'score': k}) for k,v in data.items()}

In [44]:
score_df = reduce(lambda acc,x: acc.merge(x, on='input_md5'), data.values())
score_df = score_df.merge(df, on='input_md5')
score_df['count'] = 1

In [45]:
score_df.groupby('dataset').agg({'count': 'count'}).T

dataset,事实抽取,信息定位,推理分析,推理计算,表格计算
count,29,10,12,50,30


In [46]:
score_df.groupby(['dataset']).agg({k:'mean' for k in data.keys()}).T

dataset,事实抽取,信息定位,推理分析,推理计算,表格计算
model_1,0.758621,0.7,1.0,0.68,0.633333
model_2,0.758621,1.0,1.0,0.76,0.733333


In [47]:
score_df.groupby(['dataset', 'split']).agg({'count': 'count'}).T

dataset 事实抽取                              信息定位     推理分析           推理计算       \
split   内容抽取 列举 复杂指令 实体信息抽取 数值信息抽取 结构信息抽取  位置类 引用类 因果分析 对比分析 评价分析 对比排序 数值计算   
count      5  1    2      2     16      3    1   9    6    2    4    8   42   

dataset 表格计算      
split    查询类 比较类  
count     20  10

In [48]:
score_df.groupby(['dataset', 'split']).agg({k:'mean' for k in data.keys()}).T

dataset 事实抽取                                    信息定位           推理分析            \
split   内容抽取   列举 复杂指令 实体信息抽取  数值信息抽取    结构信息抽取  位置类       引用类 因果分析 对比分析 评价分析   
model_1  1.0  1.0  0.5    1.0  0.6875  0.666667  0.0  0.777778  1.0  1.0  1.0   
model_2  1.0  1.0  0.5    1.0  0.7500  0.333333  1.0  1.000000  1.0  1.0  1.0   

dataset   推理计算            表格计算       
split     对比排序      数值计算   查询类  比较类  
model_1  0.625  0.690476  0.65  0.6  
model_2  0.750  0.761905  0.75  0.7